In [339]:
import os
import polars as pl
import altair as alt
import warnings

pl.Config(tbl_rows=100)

df = pl.read_parquet(os.path.join("data","fdb.parquet"))
vanocni = pl.read_json(os.path.join("data","vanocni_filmy_kompletni_metadata.json"))

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings("ignore")

In [340]:
vanocni_filmy = vanocni.select(pl.col("nazev")).to_series().to_list()

In [341]:
vanocni_filmy.append("Císařův pekař")
vanocni_filmy.append("Pekařův císař")

In [342]:
vanocni_filmy.sort()

In [343]:
len(vanocni_filmy)

32

In [344]:
vanocni_filmy

['Adéla ještě nevečeřela',
 'Anděl Páně',
 'Bohouš',
 'Byl jednou jeden král...',
 'Chobotnice z II. patra',
 'Císařův pekař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dařbuján a Pandrhola',
 'Dobrý voják Švejk',
 'Grinch',
 'Láska nebeská',
 'Mrazík',
 'Na samotě u lesa',
 'Nesmrtelná teta',
 'Někdo to rád horké',
 'Pekařův císař',
 'Pelíšky',
 'Pretty Woman',
 'Princ a Večernice',
 'Princezna se zlatou hvězdou',
 'Prázdniny v Římě',
 'Pyšná princezna',
 'S tebou mě baví svět',
 'S čerty nejsou žerty',
 'Shrek',
 'Skála',
 'Sám doma',
 'Sám doma 2: Ztracen v New Yorku',
 'Tři oříšky pro Popelku',
 'Veselé Vánoce přejí chobotnice',
 'Z pekla štěstí',
 'Šíleně smutná princezna']

In [345]:
do_grafu = df.filter(
    ~pl.col('nazev').is_in(['Šmankote, babičko, čaruj!','Císař a tambor','Dr. Dolittle 2','Král Králů','Duch nad zlato','Dunston: Sám v hotelu','Panna a netvor','RumplCimprCampr','O medvědu Ondřejovi','O kominickém učni a dceři cukráře'])
).filter(
        pl.col("kanal").is_in(["ČT1","ČT2","NOVA","Prima"])
    ).filter(
        pl.col("nazev").is_in(vanocni_filmy)
    ).with_columns(
        pl.col("nazev").str.replace("Dalskabáty, hříšná ves aneb Zapomenutý čert","Dalskabáty, hříšná ves").replace("Sám doma 2: Ztracen v New Yorku","Sám doma 2").replace("Veselé Vánoce přejí chobotnice","Veselé Ván. přejí chobotnice").replace('O princezně Jasněnce a létajícím ševci', 'O princ. Jasněnce a lét. ševci').replace("O kominickém učni a dceři cukráře","O komin. učni a dceři cukráře")
    ).filter(
        ((pl.col("datum").dt.month() == 12) & (pl.col("datum").dt.day() >= 22)) | ((pl.col("datum").dt.month() == 1) & (pl.col("datum").dt.day() <= 2))
    ).with_columns(
        (pl.col('datum').dt.month().cast(str) + pl.lit("-") + pl.col("datum").dt.day().cast(str)).str.replace('1-','13-').alias("kolikateho")
    ).with_columns(
        pl.when(pl.col('datum').dt.month() == 1).then(pl.col('datum').dt.day() + 31).otherwise(pl.col("datum").dt.day()).alias("kolikateho")
    ).group_by(
        ["nazev","kolikateho"]
    ).len(
    ).with_columns(
        (pl.when(
            pl.col('kolikateho') <= 31
        ).then(
            pl.col('kolikateho').cast(str) + pl.lit('.')
        ).otherwise(
            (pl.col('kolikateho') - 31).cast(str) + pl.lit('.')
        )).alias('kolikateho_str')
    )

poradi2 = do_grafu.sort("len", descending=True).unique(subset="nazev",keep="first").with_columns(pl.lit('max').alias('maximum'))

poradi_dat = [f'{x}.' for x in [y for y in range(6,32)] + [y for y in range(1,6)]]

do_grafu = do_grafu.join(poradi2, on=['nazev','kolikateho'], how='left').sort(by=['nazev','kolikateho']).fill_null(0).with_columns(
    pl.when(pl.col('maximum').is_null()).then(pl.lit('ne')).otherwise(pl.lit('max')).alias('maximum')
)

poradi2 = poradi2.sort(by=["kolikateho","len"],descending=[False,True]).select(pl.col("nazev")).to_series().to_list()

poradi3 = do_grafu.group_by("nazev").agg(
    weighted_avg_kolikateho=(pl.col("kolikateho") * pl.col("len")).sum() / pl.col("len").sum()
).sort(
    by="weighted_avg_kolikateho"
).select(
    pl.col("nazev")
).to_series().to_list()

poradi3[0:5]

['Anděl Páně', 'Grinch', 'Pelíšky', 'Chobotnice z II. patra', 'Láska nebeská']

In [346]:
df_cary = pl.DataFrame(
    {'kolikateho_str':['24.','31.'],'umisteni':poradi3[0]}
)
df_text = pl.DataFrame(
    {'kolikateho_str': ['27.'],'umisteni':poradi3[0]}
)

cary = alt.Chart(
    df_cary
).mark_rule(
    color='#A9AAC0',
    strokeWidth=16,
    opacity=0.3
).encode(
    alt.X('kolikateho_str:N',sort=poradi_dat)
)

text = alt.Chart(
    df_text
).mark_text(
    text=['← Štědrý den⠀⠀⠀⠀Silvestr →'],
    color='black',#'#B25342',
    dx=10,
    dy=-278,
    baseline='middle',
    fontSize=9,
    fontWeight=300,
    font='Asap'
).encode(
    alt.X('kolikateho_str:N',sort=poradi_dat),
    # alt.Y('umisteni:N',sort=poradi3,title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5'))
)

vanocni_obsah = (cary + text + alt.Chart(
    do_grafu.to_pandas(),
    width=240,
    title=alt.Title(
        "Vánoční filmy a kdy typicky běží",
        subtitle=[
            'Snímky jsou řazené od těch vysílaných obvykle dříve po ty vysílané později.',
            'Velikost koleček odpovídá počtu repríz, nejčastější den reprízy je červený.'
        ]
    )
).mark_circle(
    opacity=1
).encode(
    alt.X(
        'kolikateho_str:N', # do_grafu2.sort(by='kolikateho').select(pl.col('kolikateho')).to_series().to_list(),
        sort=poradi_dat,
        title=None,
        axis=alt.Axis(orient='top',labelAngle=0,labelAlign='center',tickColor='white')
    ),
    alt.Y('nazev:N',sort=poradi3,title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5')),
    alt.Size('len:Q',legend=None),
    alt.Color("maximum:N",legend=None,scale=alt.Scale(range=['#B25342','#D6B599']))
)
).configure_view(
    stroke='transparent'
)

me_to_neurazi(graf=vanocni_obsah, soubor="vanocni_obsah", kredity="vizualizace: iROZHLAS.cz | televizní programy: fdb.cz", slozka_na_serveru="grafy-tv")

vanocni_obsah

<figure>
    <a href="https://data.irozhlas.cz/grafy-tv/vanocni_obsah.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-tv/vanocni_obsah.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


alt.LayerChart(...)

In [347]:
def najdi_cloveka(clovek:str,hezky:str,kanaly=["ČT1","ČT2","NOVA","Prima","Prima Cool","Prima LOVE"]):
    return pl.DataFrame({'kolikateho':range(6,37)}).join(
        df.filter(
         pl.col("kanal").is_in(kanaly)
    ).filter(
        pl.col("extra").str.contains(clovek) | pl.col("noticka").str.contains(clovek) | pl.col("nazev").str.contains(clovek)
    ).filter(
        ((pl.col("datum").dt.month() == 12) & (pl.col("datum").dt.day() >= 6)) | ((pl.col("datum").dt.month() == 1) & (pl.col("datum").dt.day() <= 5))
    ).with_columns(
            pl.when(pl.col('datum').dt.month() == 1).then(pl.col('datum').dt.day() + 31).otherwise(pl.col("datum").dt.day()).alias("kolikateho")
    ).group_by(
        "kolikateho"
    ).len(),
    how='left',
    on='kolikateho').fill_null(0).sort(by="kolikateho").with_columns(
            (pl.when(
                pl.col('kolikateho') <= 31
            ).then(
                pl.col('kolikateho').cast(str) + pl.lit('.')
            ).otherwise(
                (pl.col('kolikateho') - 31).cast(str) + pl.lit('.')
            )).alias('kolikateho_str')
        ).with_columns(
        pl.lit(hezky).alias("kdo")
        )

In [348]:
def najdi_cloveka_celorok(clovek:str,hezky:str,kanaly=["ČT1","ČT2","NOVA","Prima","Prima Cool","Prima LOVE"]):
    return df.filter(
         pl.col("kanal").is_in(kanaly)
    ).filter(
        pl.col("extra").str.contains(clovek) | pl.col("noticka").str.contains(clovek)
    ).group_by(
        pl.col("datum").dt.month()
    ).len().sort(by="datum").with_columns(
        pl.lit(hezky).alias("kdo")
        )

In [349]:
do_grafu3 = pl.concat(
    [
        najdi_cloveka("Bohdalov","Bohdalová"),
        najdi_cloveka("Donutil","Donutil"),
        najdi_cloveka("Polívk","Polívka"),
        najdi_cloveka("Menšík","Menšík"),
        najdi_cloveka("Janžurov","Janžurová"),
        najdi_cloveka("Filipovs","Filipovský"),
        najdi_cloveka("Růžičkov","Růžičková"),
        najdi_cloveka("Kretschmerov","Kretschmerová"),
        najdi_cloveka("Šafránkov","Šafránková"),
    ]
)

alt.Chart(
    do_grafu3
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [350]:
do_grafu3 = pl.concat(
    [
        najdi_cloveka("komedi","komedie"),
        najdi_cloveka("drama","drama"),
        najdi_cloveka("zábavn","zábava"),
        najdi_cloveka("pohádk","pohádka"),
        najdi_cloveka("hudební","hudební"),
        najdi_cloveka("romantick","romantický"),
        najdi_cloveka("válečn","válečný"),
        najdi_cloveka("historick","historický"),
        najdi_cloveka("církevn","církevní"),
        najdi_cloveka("bibl","biblický"),
        najdi_cloveka("animovan","animovaný"),
        najdi_cloveka("detektiv","detektivní"),
    ]
)

alt.Chart(
    do_grafu3
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [351]:
vanocni.sort(by="nazev")

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/5981-adela-jeste-nevecerela/prehled/""","""film""","""5981-adela-jeste-nevecerela""","""Adéla ještě nevečeřela""","[""komedie"", ""krimi"", ""sci-fi""]","""Československo""",1978,105,"[""Oldřich Lipský""]","[""Jiří Brdečka"", ""Oldřich Lipský""]","[""Michal Dočolomanský"", ""Rudolf Hrušínský"", … ""více""]","""Neporazitelný soukromý detektiv Nick Carter přijíždí do Prahy řešit případ, s nímž si zdejší policie neví rady. Je vyzbrojen geniálním mozkem a nejnov…",85,40792
"""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film""","""215414-andel-pane""","""Anděl Páně""","[""pohádka""]","""Česko""",2005,91,"[""Jiří Strach""]","[""Lucie Konášová""]","[""Ivan Trojan"", ""Jiří Dvořák"", … ""více""]","""Je časné ráno, v předvečer Štědrého dne, a celé nebe se chystá na velkolepé oslavy Ježíškových narozenin. Panna Marie (Klára Issová) se svatou Veronik…",77,40797
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88,7462
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191
"""https://www.csfd.cz/film/7388-chobotnice-z-ii-patra/prehled/""","""tv""","""7388-chobotnice-z-ii-patra""","""Chobotnice z II. patra""","[""rodinný"", ""komedie"", ""sci-fi""]","""Československo / Západní Německo""",1986,92,"[""Jindřich Polák""]","[""Ota Hofman""]","[""Pavel Zedníček"", ""Dagmar Havlová"", … ""více""]","""Eva a Honzík si pro zábavu vymodelují z neznámé hmoty dvě chobotničky. Nic netuší, že hmota je živá a dokonce mluví. Kolem nich se začnou dít prapodiv…",66,17426
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapomenutého čerta, aby z n…",80,3101
"""https://www.csfd.cz/film/3095-darbujan-a-pandrhola/prehled/""","""film""","""3095-darbujan-a-pandrhola""","""Dařbuján a Pandrhola""","[""pohádka"", ""komedie""]","""Československo""",1959,80,"[""Martin Frič""]","[""Jan Drda"", ""Martin Frič""]","[""Jiří Sovák"", ""Rudolf Hrušínský"", … ""více""]","""Hrdinou je chudý havíř Kuba Dařbuján, který se stará o svou Markýtku a jedenáct dětí. Není divu, že se jen málokdy nají dosyta. A to jim vrána právě n…",83,27649
"""https://www.csfd.cz/film/8668-dobry-vojak-svejk/prehled/""","""film""","""8668-dobry-vojak-svejk""","""Dobrý voják Švejk""","[""komedie"", ""válečný""]","""Československo""",1956,105,"[""Karel Steklý""]","[""Karel Steklý""]","[""Rudolf Hrušínský"", ""Eva Svobodová"", … ""více""]","""Kinematografií se už od dob němého filmu táhne pestrá nit pokusů o více či měně věrné adaptace tohoto Haškova slavného románu. Mezi nejznámější patří …",86,43087
"""https://www.csfd.cz/film/4551-grinch/pr

In [352]:
vanocni.explode("hraji").group_by("hraji").len().sort(by="len",descending=True)

hraji,len
str,u32
"""více""",29
"""Květa Fialová""",6
"""Jaroslav Moučka""",5
"""František Filipovský""",4
"""Oldřich Vlach""",4
"""Miloš Vavruška""",4
"""Naďa Konvalinková""",4
"""Václav Švec""",4
"""Josef Kemr""",4


In [353]:
do_grafu4 = pl.concat(
    [
        najdi_cloveka_celorok("Bohdalov","Bohdalová"),
        najdi_cloveka_celorok("Donutil","Donutil"),
        najdi_cloveka_celorok("Polívk","Polívka"),
        najdi_cloveka_celorok("Menšík","Menšík"),
        najdi_cloveka_celorok("Janžurov","Janžurová"),
        najdi_cloveka_celorok("Filipovs","Filipovský"),
        najdi_cloveka_celorok("Růžičkov","Růžičková"),
        najdi_cloveka_celorok("Kretschmerov","Kretschmerová"),
        najdi_cloveka_celorok("Šafránkov","Šafránková")
    ]
)

alt.Chart(
    do_grafu4,
    width=600,
    height=50
).mark_bar(
).encode(
    alt.X("datum:Q",sort=poradi_dat),
    alt.Y("len:Q"),
    alt.Row("kdo:N")
)

alt.Chart(...)

In [354]:
do_grafu5 = pl.concat(
    [
        najdi_cloveka("Hitler","Hitler",kanaly=['ČT2']),
        najdi_cloveka("Ježíš","Ježíš",kanaly=['ČT2']),
    #    najdi_cloveka("Stalin","Stalin"),
    #    najdi_cloveka("Masaryk","Masaryk"),
    #    najdi_cloveka("Havel","Havel"),
    ]
)

alt.Chart(
    do_grafu5,
    width=600
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [355]:
df.filter(pl.col("kanal") == "NOVA").select(pl.col("datum").max())

datum
datetime[μs]
2025-04-17 23:50:00


In [356]:
do_grafu5 = pl.concat(
    [
        najdi_cloveka("(Hitler|Stalin|Mussolini)","Hitler, Stalin, Mussolini",kanaly=['ČT1','ČT2',"NOVA","Prima"]),
        najdi_cloveka("(Ježíš|Kristu|Kristov)","Ježíš",kanaly=['ČT1','ČT2',"NOVA","Prima"]),

    #    najdi_cloveka("Stalin","Stalin"),
    #    najdi_cloveka("Masaryk","Masaryk"),
    #    najdi_cloveka("Havel","Havel"),
    ]
)

jezis = alt.Chart(
    do_grafu5.filter((pl.col("kolikateho") <= 31) & (pl.col("kolikateho") >= 10)),
    title=alt.Title(
        "Týden, kdy Ježíš střídá diktátory",
        subtitle=["Náměty pořadů vysílaných na ČT1, ČT2, Nově a Primě v prosincích 2007-24."]
    ),
    width=320,
    height=50
).mark_bar(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat,title=None,axis=alt.Axis(grid=False, gridColor='#E5E5E5',labelAngle=0,labelAlign='center',tickColor='#E5E5E5')),
    alt.Y("len:Q",title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5',labelExpr="datum.label + ' pořadů'", orient='right')),
    alt.Color("kdo:N",scale=alt.Scale(range=['#D6B599','#B25342']), legend=alt.Legend(orient='top',title=None))
).configure_view(
    stroke='transparent'
)

jezis

alt.Chart(...)

In [357]:
jezis = alt.Chart(
    do_grafu5.filter((pl.col("kolikateho") <= 31) & (pl.col("kolikateho") >= 10)),
    title=alt.Title(
        "Týden, kdy Ježíš střídá diktátory",
        subtitle=["Náměty pořadů vysílaných na ČT1, ČT2, Nově a Primě v prosincích 2007-24."]
    ),
    width=320,
    height=50
).mark_bar().encode(
    alt.X("kolikateho_str:N", sort=poradi_dat, title=None, axis=alt.Axis(grid=False, gridColor='#E5E5E5', labelAngle=0, labelAlign='center', tickColor='#E5E5E5')),
    alt.Y("len:Q", title=None, axis=alt.Axis(grid=True, gridColor='#E5E5E5', tickColor='#E5E5E5', tickCount=3, labelExpr="datum.label + ' pořadů'", orient='right')),
    alt.Color("kdo:N", scale=alt.Scale(range=['#D6B599', '#B25342']), legend=alt.Legend(orient='top', title=None)),
    
    # This line creates the side-by-side effect
    alt.XOffset("kdo:N") 
).configure_view(
    stroke='transparent'
)

jezis

alt.Chart(...)

In [358]:
me_to_neurazi(graf=jezis, soubor='jezis', kredity="vizualizace: iROZHLAS.cz | televizní programy: fdb.cz", slozka_na_serveru="grafy-tv")

<figure>
    <a href="https://data.irozhlas.cz/grafy-tv/jezis.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-tv/jezis.svg" width="100%" alt="Graf s titulkem „Týden, kdy Ježíš střídá diktátory“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [359]:
vanocni.explode("hraji").group_by('hraji').len().sort(by='len',descending=True)

hraji,len
str,u32
"""více""",29
"""Květa Fialová""",6
"""Jaroslav Moučka""",5
"""Mirko Musil""",4
"""Miloš Kopecký""",4
"""František Filipovský""",4
"""Václav Švec""",4
"""Stanislav Neumann""",4
"""Miloš Vavruška""",4
